In [2]:
import cupy
import cudf
import cuml

In [4]:
df=cudf.read_csv('USA_Housing.csv')

In [5]:
df.head

<bound method DataFrame.head of       Avg. Area Income  Avg. Area House Age  Avg. Area Number of Rooms  \
0          79545.45857             5.682861                   7.009188   
1          79248.64245             6.002900                   6.730821   
2          61287.06718             5.865890                   8.512727   
3          63345.24005             7.188236                   5.586729   
4          59982.19723             5.040555                   7.839388   
...                ...                  ...                        ...   
4995       60567.94414             7.830362                   6.137356   
4996       78491.27543             6.999135                   6.576763   
4997       63390.68689             7.250591                   4.805081   
4998       68001.33124             5.534388                   7.130144   
4999       65510.58180             5.992305                   6.792336   

      Avg. Area Number of Bedrooms  Area Population         Price  
0          

In [6]:
df.info()

<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 6 columns):
 #   Column                        Non-Null Count  Dtype
---  ------                        --------------  -----
 0   Avg. Area Income              5000 non-null   float64
 1   Avg. Area House Age           5000 non-null   float64
 2   Avg. Area Number of Rooms     5000 non-null   float64
 3   Avg. Area Number of Bedrooms  5000 non-null   float64
 4   Area Population               5000 non-null   float64
 5   Price                         5000 non-null   float64
dtypes: float64(6)
memory usage: 234.4 KB


In [8]:
%%time
df=df.sample(frac=1).reset_index(drop=True)

CPU times: user 2.03 ms, sys: 284 µs, total: 2.31 ms
Wall time: 1.55 ms


In [9]:
%%time
X=df.iloc[:,:5]
Y=df.iloc[:,5]

CPU times: user 2.11 ms, sys: 586 µs, total: 2.69 ms
Wall time: 2.44 ms


In [11]:
%%time
from cuml.preprocessing import StandardScaler
ss=StandardScaler()
X_scaled=ss.fit_transform(X)
X_scaled.insert(loc=0,value=1.0,name='Ones')
X_scaled

CPU times: user 340 ms, sys: 15.9 ms, total: 356 ms
Wall time: 355 ms


,Ones,0,1,2,3,4
0,1.0,-0.057285,-0.484743,-0.646798,-0.616954,-0.836118
1,1.0,-1.892183,-0.122752,-0.872681,0.047544,0.539050
2,1.0,-0.247181,-1.987382,1.855939,-0.714198,0.513610
3,1.0,0.703111,-0.207309,0.839329,-0.552125,0.923363
4,1.0,1.071384,-0.367598,-1.071636,-0.657472,0.612810
...,...,...,...,...,...,...
4995,1.0,-1.647482,-1.118030,0.215477,0.063751,-1.240347
4996,1.0,0.128234,0.857907,-1.114892,0.347379,0.842100
4997,1.0,0.972947,-0.538840,-1.221208,-1.459732,-0.514725
4998,1.0,0.362834,0.057687,-0.267299,0.387897,0.796269


In [18]:
%%time
X_divided=[X_scaled[0:1000],X_scaled[1000:2000],X_scaled[2000:3000],X_scaled[3000:4000],X_scaled[4000:5000]]
Y_divided=[Y[0:1000],Y[1000:2000],Y[2000:3000],Y[3000:4000],Y[4000:5000]]

CPU times: user 2.21 ms, sys: 0 ns, total: 2.21 ms
Wall time: 2.15 ms


In [19]:
%%time
beta_values=[]
R2_values=[]
maxR2=0
max_index=0
for i in range(0,5):
    trainX=[]
    trainY=[]
    for j in range(0,5):
        if j!=i:
            trainX.append(X_divided[j])
            trainY.append(Y_divided[j])
    X_train=cudf.concat(trainX,axis=0)
    Y_train=cudf.concat(trainY,axis=0)
    X_test=X_divided[i]
    Y_test=Y_divided[i]
    A=X_train.values.T.dot(X_train.values)
    B=cupy.linalg.inv(A)
    C=B.dot(X_train.values.T)
    beta=C.dot(Y_train.values)
    Y_predict=X_test.values.dot(beta)
    beta_values.append(beta)
    error=Y_predict-Y_test.values
    sse=cupy.sum(error**2)
    y_mean=cupy.mean(Y_test)
    total_variance=cupy.sum((Y_test.values-y_mean)**2)
    r2=1-sse/total_variance
    R2_values.append(r2)
    if(r2>maxR2):
        maxR2=r2
        max_index=i
print(beta_values)
print(R2_values)
print(max_index)

[array([1231404.68195074,  230617.20864979,  165482.32432908,
        120105.38586486,    2446.08479462,  150066.40345121]), array([1232190.16198702,  228075.62456364,  163341.45319771,
        121558.20015731,    2085.37373344,  150556.96282067]), array([1.23075565e+06, 2.30312779e+05, 1.65065115e+05, 1.22571973e+05,
       7.00910620e+02, 1.51970590e+05]), array([1232234.59152172,  229564.06053859,  163911.44313782,
        120630.1250546 ,    2024.08962387,  150968.97439235]), array([1233766.99491969,  231244.72482446,  163282.00943295,
        121829.77902768,    2985.6913637 ,  150823.08995367])]
[array(0.91781252), array(0.9157366), array(0.91780696), array(0.92015321), array(0.91679183)]
3
CPU times: user 1.15 s, sys: 15.3 ms, total: 1.16 s
Wall time: 1.16 s


In [20]:
%%time
betaMax=beta_values[max_index]

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.01 µs


In [21]:
betaMax

array([1232234.59152172,  229564.06053859,  163911.44313782,
        120630.1250546 ,    2024.08962387,  150968.97439235])

In [22]:
%%time
from cuml.model_selection import train_test_split
X_train1,X_test1,Y_train1,Y_test1=train_test_split(X_scaled,Y,test_size=0.30,random_state=42)

CPU times: user 805 ms, sys: 3.27 ms, total: 809 ms
Wall time: 808 ms


In [23]:
%%time
Y_predict=X_test1.values.dot(betaMax)
error=Y_predict-Y_test1.values
sse=cupy.sum(error**2)
total_variance=cupy.sum((Y_test1.values-y_mean)**2)
r2_final=1-sse/total_variance
print(r2_final)

0.9204524967946758
CPU times: user 2.63 ms, sys: 186 µs, total: 2.82 ms
Wall time: 2.07 ms


In [38]:
%%time
import numpy as np
import pandas as pd
df1=pd.read_csv('USA_Housing.csv')
X1=df1.iloc[:,:5]
Y1=df1.iloc[:,5]
from sklearn.preprocessing import StandardScaler
ss=StandardScaler()
X_scaled1=ss.fit_transform(X1)

CPU times: user 7.24 ms, sys: 0 ns, total: 7.24 ms
Wall time: 6.71 ms


In [39]:
%%time
from sklearn.model_selection import train_test_split
X_train2,X_test2,Y_train2,Y_test2=train_test_split(X_scaled1,Y1,test_size=0.25,random_state=42)

CPU times: user 1.59 ms, sys: 0 ns, total: 1.59 ms
Wall time: 1.1 ms


In [41]:
%%time
from sklearn.linear_model import LinearRegression
lr=LinearRegression()
lr.fit(X_train2,Y_train2)
print(lr.score(X_test2,Y_test2))

0.9169678829400091
CPU times: user 1.9 ms, sys: 0 ns, total: 1.9 ms
Wall time: 1.41 ms


In [42]:
%%time
from cuml.linear_model import LinearRegression
Lr=LinearRegression()
Lr.fit(X_train2,Y_train2)
print(Lr.score(X_test2,Y_test2))

0.9169678829400092
CPU times: user 216 ms, sys: 0 ns, total: 216 ms
Wall time: 215 ms
